In [139]:
impact.df = read.table("../../../data/all_IMPACT_mutations_180508.txt", sep="\t", stringsAsFactors=F, header=T)
annotation.df <- read.table("../../../data/all_IMPACT_mutations_180508.simple.hg19_multianno.txt", sep="\t", stringsAsFactors=F, header=T)

In [156]:
impact.ann.df <- cbind(impact.df, annotation.df )
impact.unlikely.df <- subset(impact.ann.df, confidence_class=='UNLIKELY')[1:2000,]
impact.drivers.df <- subset(impact.ann.df, confidence_class %in% c('MANUAL_OK', 'AUTO_OK'))[1:2000,]
impact.ann.df <- rbind(impact.unlikely.df, impact.drivers.df )

In [157]:
nrow(impact.unlikely.df)

[1] 2000

In [158]:
nrow(impact.drivers.df)

[1] 2000

In [159]:
source('parseCosmic.R')
impact.ann.df <- parseCosmic(impact.ann.df)

In [160]:
cg <- read.table('../../../data/CancerGenesList.txt', sep='\t', header=TRUE, comment.char = '', row.names = 1)
impact.ann.df$isOncogene <- cg[impact.ann.df$Hugo_Symbol,'OncoKB.Oncogene']
impact.ann.df$cgType[impact.ann.df$isOncogene=='Yes'] <- 'Oncogene'
impact.ann.df$cgType[!impact.ann.df$isOncogene=='Yes'] <- 'Recessive'
impact.ann.df$cgType[is.na(impact.ann.df$cgType) ] <- 'Unknown'
impact.ann.df$cgType <- as.factor(impact.ann.df$cgType)

table(impact.ann.df$cgType)
sum(is.na(impact.ann.df$cgType))


 Oncogene Recessive   Unknown 
     1134      2691       175 

[1] 0

In [161]:
table(impact.ann.df$occurence_in_normals)


   0 
4000 

In [162]:
head(impact.ann.df )

,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Consequence,Variant_Type,⋯,phastCons20way_mammalian,SiPhy_29way_logOdds,X1000g2015aug_all,Kaviar_AF,Kaviar_AC,Kaviar_AN,cosmic70,cosmicCount,isOncogene,cgType
2,ALK,0,MSK-IMPACT,hg19,2,29543736,29543736,+,nonsynonymous_SNV,SNP,⋯,0.336,6.380,0.0159744,0.0300125,4640,154602,ID=COSM3758202;OCCURENCE=1(large_intestine),1,Yes,Oncogene
3,PDCD1,0,MSK-IMPACT,hg19,2,242793433,242793433,+,nonsynonymous_SNV,SNP,⋯,0.002,7.501,0.136781,0.0760792,11762,154602,.,0,,Recessive
4,MAP3K1,0,MSK-IMPACT,hg19,5,56177843,56177843,+,nonsynonymous_SNV,SNP,⋯,0.984,13.098,0.00738818,0.0141072,2181,154602,.,0,,Recessive
5,FLT4,0,MSK-IMPACT,hg19,5,180030313,180030313,+,nonsynonymous_SNV,SNP,⋯,0.001,2.012,.,0.0789964,12213,154602,.,0,Yes,Oncogene
6,FLT4,0,MSK-IMPACT,hg19,5,180051003,180051003,+,nonsynonymous_SNV,SNP,⋯,0.281,5.808,0.0529153,0.0911566,14093,154602,.,0,Yes,Oncogene
7,NOTCH4,0,MSK-IMPACT,hg19,6,32178570,32178570,+,nonsynonymous_SNV,SNP,⋯,0.055,15.605,0.0449281,0.0186802,2888,154602,.,0,,Recessive


In [163]:
write.table(impact.ann.df, file="../../../data/all_IMPACT_mutations_180508.annot.csv", sep=',', quote=FALSE, row.names=FALSE)

In [165]:
labelTrain <- impact.ann.df$confidence_class=='MANUAL_OK' | impact.ann.df$confidence_class=='AUTO_OK'
length(labelTrain)

[1] 4000

In [28]:
#colnames(impact.sel.df)

In [166]:
selFeatures <- c('Consequence',
                 'cgType',
                #'t_depth',
                 't_vaf',
                 't_alt_count',
                 'n_depth',
                 #'n_vaf',
                 #'n_alt_count',
                 't_ref_plus_count',
                 't_ref_neg_count',
                 #'minor_contamination',
                 'SIFT_score',
                 #'Polyphen2_HDIV_score',
                 #'LRT_score',
                 #'MutationTaster_score',
                 #'MutationAssessor_score',
                 #'FATHMM_score',
                 #'PROVEAN_score',
                 #'VEST3_score',
                 #'CADD_raw',
                 #'DANN_score',
                 #'fathmm.MKL_coding_score',
                 #'MetaSVM_score',
                 #'MetaLR_score',
                 #'integrated_fitCons_score',
                 #'X1000g2015aug_all',
                 'Kaviar_AF',
                 'cosmicCount'
                )
variantTrain <- impact.sel.df

In [179]:
variantTrain <- impact.sel.df[, selFeatures]
nrow(variantTrain)

[1] 4000

In [174]:
#sapply(variantTrain, class)

In [180]:
variantTrain.corr <- variantTrain
variantTrain.corr$Consequence <- as.factor(variantTrain.corr$Consequence)
variantTrain.corr$t_vaf <- as.numeric(variantTrain.corr$t_vaf)
variantTrain.corr$t_alt_count <- as.numeric(variantTrain.corr$t_alt_count)
variantTrain.corr$n_depth <- as.numeric(variantTrain.corr$n_depth)
variantTrain.corr$t_ref_plus_ratio<- as.numeric(variantTrain.corr$t_ref_plus_count)  / 
(as.numeric(variantTrain.corr$t_ref_plus_count) + as.numeric(variantTrain.corr$t_ref_neg_count))

variantTrain.corr$SIFT_score<- as.numeric(variantTrain.corr$SIFT_score)
variantTrain.corr$Kaviar_AF<- as.numeric(variantTrain.corr$Kaviar_AF)

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”

In [181]:
variantTrain.corr$SIFT_score[is.na(variantTrain.corr$SIFT_score)] <- 1

In [182]:
variantTrain.corr$t_ref_plus_ratio[is.na(variantTrain.corr$t_ref_plus_ratio)] <- 0.5

In [183]:
variantTrain.corr$Kaviar_AF[is.na(variantTrain.corr$Kaviar_AF)] <- 0

In [184]:
table(variantTrain.corr$cgType)
sum(is.na(variantTrain.corr))


 Oncogene Recessive   Unknown 
     1134      2691       175 

[1] 0

In [185]:
colSums(is.na(variantTrain.corr))

Consequence           cgType            t_vaf      t_alt_count 
               0                0                0                0 
         n_depth t_ref_plus_count  t_ref_neg_count       SIFT_score 
               0                0                0                0 
       Kaviar_AF      cosmicCount t_ref_plus_ratio 
               0                0                0

In [186]:
save(variantTrain.corr, labelTrain, file='../../../data/trainingData.RData')

In [176]:
head(variantTrain.corr)

,Consequence,t_vaf,t_alt_count,n_depth,t_ref_plus_count,t_ref_neg_count,SIFT_score,Kaviar_AF,cosmicCount,t_ref_plus_ratio
2,nonsynonymous_SNV,0.06736,13,793,78,102,0.539,0.0300125,1,0.4333333
3,nonsynonymous_SNV,0.06250,15,712,89,136,0.019,0.0760792,0,0.3955556
4,nonsynonymous_SNV,0.03141,12,666,200,170,0.002,0.0141072,0,0.5405405
5,nonsynonymous_SNV,0.06494,25,1112,182,178,0.650,0.0789964,0,0.5055556
6,nonsynonymous_SNV,0.07458,22,977,116,157,0.225,0.0911566,0,0.4249084
7,nonsynonymous_SNV,0.05743,17,780,161,118,0.001,0.0186802,0,0.5770609
